In [3]:
import csv
import random

class Buffer:
    def __init__(self, occupancy_threshold=0.9):
        self.entries = {}
        self.occupancy_threshold = occupancy_threshold

    def set_max_buffer_size(self, buffer_id, num_entries):
        self.entries[buffer_id] = [None] * num_entries

    def get_buffer_occupancy(self, buffer_id):
        occupancy = len([entry for entry in self.entries[buffer_id] if entry is not None])
        return occupancy / len(self.entries[buffer_id])

class Arbiter:
    def __init__(self):
        self.weights = {}

    def set_arbiter_weights(self, agent_type, weight):
        self.weights[agent_type] = weight

    def get_arb_rates(self, agent_type):
        return self.weights.get(agent_type, 0)

class Memory:
    def __init__(self):
        self.storage = {}

    def read(self, address):
        return self.storage.get(address, None)

    def write(self, address, data):
        self.storage[address] = data

class Interface:
    def __init__(self, bandwidth=32, max_bandwidth=32):
        self.bandwidth = bandwidth  # bytes per cycle
        self.max_bandwidth = max_bandwidth

    def transfer_data(self, size):
        return size / self.bandwidth

class CPU:
    def __init__(self, interface, memory, buffer):
        self.interface = interface
        self.memory = memory
        self.buffer = buffer

    def read_memory(self, address):
        return self.memory.read(address)

    def write_memory(self, address, data):
        self.memory.write(address, data)

class IO:
    def __init__(self, interface, memory, buffer):
        self.interface = interface
        self.memory = memory
        self.buffer = buffer

    def read_memory(self, address):
        return self.memory.read(address)

    def write_memory(self, address, data):
        self.memory.write(address, data)

class Simulator:
    def __init__(self, min_latency, max_bandwidth):
        self.memory = Memory()
        self.interface = Interface(max_bandwidth=max_bandwidth)
        self.cpu_buffer = Buffer()
        self.io_buffer = Buffer()
        self.arbiter = Arbiter()
        self.cpu = CPU(self.interface, self.memory, self.cpu_buffer)
        self.io = IO(self.interface, self.memory, self.io_buffer)
        self.frequency = 1.0  # Initial frequency (normalized)
        self.min_latency = min_latency
        self.max_bandwidth = max_bandwidth
        self.total_latency = 0
        self.total_bandwidth = 0
        self.num_operations = 0

    def set_max_buffer_size(self, buffer_id, num_entries):
        if buffer_id == "CPU_buffer":
            self.cpu_buffer.set_max_buffer_size(buffer_id, num_entries)
        elif buffer_id == "IO_buffer":
            self.io_buffer.set_max_buffer_size(buffer_id, num_entries)
        else:
            raise ValueError("Invalid buffer ID")

    def throttle(self):
        power_threshold = self.get_powerlimit_threshold()
        if power_threshold == 1:
            self.frequency *= 0.9  # Reduce frequency by 10%

    def simulate_operation(self, operation):
        agent, action, address, data = operation
        latency = self.measure_latency()
        bandwidth = self.measure_bandwidth()
        self.total_latency += latency
        self.total_bandwidth += bandwidth
        self.num_operations += 1
        if isinstance(address, str) and address.startswith("0x"):
            try:
                address = int(address, 16)  # Convert hexadecimal string to integer
            except ValueError:
                pass  # Ignore if conversion fails
        if agent == "CPU":
            if action == "read":
                result = self.cpu.read_memory(address)
            elif action == "write":
                self.cpu.write_memory(address, data)
                result = data  # For write operations, result is the written data
        elif agent == "IO":
            if action == "read":
                result = self.io.read_memory(address)
            elif action == "write":
                self.io.write_memory(address, data)
                result = data  # For write operations, result is the written data
        else:
            result = None
        return result, latency, bandwidth

    def get_powerlimit_threshold(self):
        # Simulate power threshold calculation (0 or 1)
        return random.choice([0, 1])

    def measure_latency(self):
        # Simulate latency measurement
        return random.randint(1, self.min_latency)

    def measure_bandwidth(self):
        # Simulate bandwidth measurement
        return random.uniform(0.95 * self.max_bandwidth, self.max_bandwidth)


# Read heavy transactions from CSV file
heavy_transactions = []
with open('heavy_transactions_1000_samples.csv', mode='r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip header row
    for row in reader:
        timestamp, txn_type, data = row
        heavy_transactions.append((int(timestamp), txn_type, data if data else None))

# Example usage
min_latency = 5  # Minimum acceptable latency
max_bandwidth = 100  # Maximum available bandwidth

sim = Simulator(min_latency, max_bandwidth)
sim.set_max_buffer_size("CPU_buffer", 10)  # Set CPU buffer size
sim.set_max_buffer_size("IO_buffer", 10)   # Set IO buffer size

# Lists to store individual and average latencies and bandwidths
individual_latencies = []
individual_bandwidths = []
total_latencies = []
total_bandwidths = []

# Simulate workload from CSV file
for timestamp, txn_type, data in heavy_transactions:
    if txn_type == "Rd":
        operation = ("CPU", "read", "Addr1", None)
    elif txn_type == "Wr":
        operation = ("CPU", "write", "Addr2" if timestamp == 2 else "Addr3", data)
    elif txn_type == "Data":
        operation = ("CPU", "write", "Addr1", data)
    
    if operation:
        result, latency, bandwidth = sim.simulate_operation(operation)
        print("Timestamp:", timestamp, "Transaction Type:", txn_type, "Result:", result, "Latency:", latency, "Bandwidth:", bandwidth)
        individual_latencies.append(latency)
        individual_bandwidths.append(bandwidth)

# Calculate average latency and bandwidth
average_latency = sim.total_latency / sim.num_operations
average_bandwidth = sim.total_bandwidth / sim.num_operations

print("Average Latency:", average_latency)
print("Average Bandwidth:", average_bandwidth)

# Throttle simulation
sim.throttle()
print("Frequency after throttling:", sim.frequency)


Timestamp: 0 Transaction Type: Rd Result: None Latency: 3 Bandwidth: 97.43905546654088
Timestamp: 10 Transaction Type: Data Result: 0d99f027 Latency: 4 Bandwidth: 98.28207120568557
Timestamp: 20 Transaction Type: Wr Result: 52c10083 Latency: 5 Bandwidth: 97.69723172991083
Timestamp: 30 Transaction Type: Rd Result: 0d99f027 Latency: 1 Bandwidth: 99.42529677278772
Timestamp: 40 Transaction Type: Wr Result: 6596cd30 Latency: 2 Bandwidth: 97.75576159885642
Timestamp: 50 Transaction Type: Wr Result: 63912107 Latency: 3 Bandwidth: 95.53366503679044
Timestamp: 60 Transaction Type: Data Result: 162e8c34 Latency: 1 Bandwidth: 97.18384518965314
Timestamp: 70 Transaction Type: Data Result: 846d569a Latency: 5 Bandwidth: 98.18096523372509
Timestamp: 80 Transaction Type: Rd Result: 846d569a Latency: 1 Bandwidth: 99.46153120748512
Timestamp: 90 Transaction Type: Wr Result: 0c19d2f4 Latency: 4 Bandwidth: 98.23322294080593
Timestamp: 100 Transaction Type: Wr Result: 673168b3 Latency: 5 Bandwidth: 99.2